# Machine Intelligence with Deep Learning
## Importance batching for improved training of neural networks
---

In [1]:
import timeit

In [2]:
from models.resnet import ResNet18
from utils.data_utils import DataLoader

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import os
import pandas as pd

from datetime import datetime
today = datetime.today().strftime('%Y%m%d')

In [3]:
SEEDS = [10, 42, 4] # don't change!

In [4]:
### Training
def train(epoch, optimizer, criterion, dataloader):
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(dataloader.yield_batches(use_train=True)):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
    train_acc = 100.*correct/total
    train_loss /= total
    return train_acc, train_loss

### Testing
def test(epoch, best_acc, seed, dataloader):
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(dataloader.yield_batches(use_train=False)):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    # Save checkpoint.
    test_acc = 100.*correct/total
    test_loss /= total
    if test_acc > best_acc:
        best_acc = test_acc
        print("==> Saving to checkpoint..")
        net.save(best_acc, epoch, seed)
    return test_acc, test_loss

In [5]:
resume = False
given_date = '20191113'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device == 'cuda':
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
### task: classification of the following classes
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

    
### hyperparameters
test_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch
num_epochs = 300 # number of iterations the model gets trained
learning_rate = 0.1 # factor for weight updates
learning_rate_switches = { # learning rate is reset after specific epochs
    '150': 0.01,
    '250': 0.001
}
momentum = 0.9
weight_decay = 5e-4

start = timeit.default_timer()

rows = []
for seed in SEEDS:
    for shuffle_setting in [{'train': False, 'test': False}]:#, {'train': True, 'test': True}]:
        print("\n==> Beginning training with seed {} and shuffle setting {}".format(seed, shuffle_setting))
        print("-" * 30)
        torch.manual_seed(seed)
        if device == 'cuda':
            torch.cuda.manual_seed_all(seed)
             
        ### Model
        print('==> Building model..')
        net = ResNet18()
        net = net.to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=momentum, weight_decay=weight_decay)
        
        ### load the data
        # if needed, specify batch sizes and shuffle settings
        print('==> Loading data..')
        dataloader = DataLoader()
        dataloader.download_cifar()
        dataloader.prepare_cifar('homogeneous', random_state=seed, batch_size=64)

        if resume:
            print('==> Resuming from checkpoint..')
            assert os.path.isdir('serialized'), 'Error: no serialized directory found!'
            ckpt = torch.load('./serialized/{}/ckpt_{}.pth'.format(given_date, seed))
            test_acc, start_epoch, net = net.load(ckpt)

        for epoch in range(start_epoch, start_epoch+num_epochs):
            if str(epoch+1) in learning_rate_switches.keys():
                print('==> Resetting learning rate')
                for param_group in optimizer.param_groups:
                    param_group['lr'] = learning_rate_switches[str(epoch+1)]
                    
            train_acc, train_loss = train(epoch, optimizer, criterion, dataloader)
            test_acc, test_loss = test(epoch, test_acc, seed, dataloader)
            print("[{}/{}]: Train Acc: {} | Test Acc: {} | Train Loss: {} | Test Loss: {}"\
                  .format(epoch+1, num_epochs, train_acc, test_acc, train_loss, test_loss))
            row = {
                'epoch': epoch + 1,
                'seed': seed,
                'train': True,
                'shuffle': True if shuffle_setting['train'] == True else False,
                'accuracy': train_acc,
                'loss': train_loss
            }
            rows.append(row)
            row = {
                'epoch': epoch + 1,
                'seed': seed,
                'train': False,
                'shuffle': True if shuffle_setting['train'] == True else False,
                'accuracy': test_acc,
                'loss': test_loss
            }
            rows.append(row)
            
stop = timeit.default_timer()
logging_df = pd.DataFrame(rows, columns=['epoch', 'seed', 'train', 'shuffle', 'accuracy', 'loss'])   
training_logs_dir = 'training_logs'
logging_df.to_csv('{}.txt'.format(os.path.join(training_logs_dir, today)), sep='\t', index=False)
print(stop - start)


==> Beginning training with seed 10 and shuffle setting {'train': False, 'test': False}
------------------------------
==> Building model..
==> Loading data..
==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified
==> Saving to checkpoint..
[1/300]: Train Acc: 85.628 | Test Acc: 10.0 | Train Loss: 0.02803756121143699 | Test Loss: 0.2070548736318946
[2/300]: Train Acc: 81.68 | Test Acc: 10.0 | Train Loss: 0.026427149031609296 | Test Loss: 0.20097469370514154
[3/300]: Train Acc: 81.68 | Test Acc: 10.0 | Train Loss: 0.026225016221106054 | Test Loss: 0.19873998127430678
[4/300]: Train Acc: 81.552 | Test Acc: 10.0 | Train Loss: 0.026128724949210883 | Test Loss: 0.19835535020828246
[5/300]: Train Acc: 81.424 | Test Acc: 10.0 | Train Loss: 0.026110236986130474 | Test Loss: 0.19830398073196412
[6/300]: Train Acc: 81.424 | Test Acc: 10.0 | Train Loss: 0.02610622972726822 | Test Loss: 0.1983169355392456
[7/300]: Train Acc: 81.424 | Test Acc: 10.0 | Trai

[72/300]: Train Acc: 81.424 | Test Acc: 10.0 | Train Loss: 0.026106308497190474 | Test Loss: 0.19837279539108277
[73/300]: Train Acc: 81.424 | Test Acc: 10.0 | Train Loss: 0.026106307830810548 | Test Loss: 0.19837279539108277
[74/300]: Train Acc: 81.424 | Test Acc: 10.0 | Train Loss: 0.026106308497190474 | Test Loss: 0.19837279539108277
[75/300]: Train Acc: 81.424 | Test Acc: 10.0 | Train Loss: 0.026106307830810548 | Test Loss: 0.19837279539108277
[76/300]: Train Acc: 81.424 | Test Acc: 10.0 | Train Loss: 0.026106308497190474 | Test Loss: 0.19837279539108277
[77/300]: Train Acc: 81.424 | Test Acc: 10.0 | Train Loss: 0.026106307830810548 | Test Loss: 0.19837279539108277
[78/300]: Train Acc: 81.424 | Test Acc: 10.0 | Train Loss: 0.026106308497190474 | Test Loss: 0.19837279539108277
[79/300]: Train Acc: 81.424 | Test Acc: 10.0 | Train Loss: 0.026106307830810548 | Test Loss: 0.19837279539108277
[80/300]: Train Acc: 81.424 | Test Acc: 10.0 | Train Loss: 0.026106308497190474 | Test Loss: 0.1

[145/300]: Train Acc: 81.424 | Test Acc: 10.0 | Train Loss: 0.026106307830810548 | Test Loss: 0.19837279539108277
[146/300]: Train Acc: 81.424 | Test Acc: 10.0 | Train Loss: 0.026106308497190474 | Test Loss: 0.19837279539108277
[147/300]: Train Acc: 81.424 | Test Acc: 10.0 | Train Loss: 0.026106307830810548 | Test Loss: 0.19837279539108277
[148/300]: Train Acc: 81.424 | Test Acc: 10.0 | Train Loss: 0.026106308497190474 | Test Loss: 0.19837279539108277
[149/300]: Train Acc: 81.424 | Test Acc: 10.0 | Train Loss: 0.026106307830810548 | Test Loss: 0.19837279539108277
==> Resetting learning rate
[150/300]: Train Acc: 38.416 | Test Acc: 10.0 | Train Loss: 0.045051663065552715 | Test Loss: 0.0600792416036129
[151/300]: Train Acc: 50.32 | Test Acc: 10.0 | Train Loss: 0.028086332674622537 | Test Loss: 0.059488871389627454
[152/300]: Train Acc: 50.064 | Test Acc: 10.0 | Train Loss: 0.028281740535497667 | Test Loss: 0.05944687328338623
[153/300]: Train Acc: 50.064 | Test Acc: 10.0 | Train Loss: 0

[217/300]: Train Acc: 50.064 | Test Acc: 10.0 | Train Loss: 0.028284370386004447 | Test Loss: 0.059444676452875135
[218/300]: Train Acc: 50.064 | Test Acc: 10.0 | Train Loss: 0.028284370386004447 | Test Loss: 0.059444676452875135
[219/300]: Train Acc: 50.064 | Test Acc: 10.0 | Train Loss: 0.028284370386004447 | Test Loss: 0.059444676452875135
[220/300]: Train Acc: 50.064 | Test Acc: 10.0 | Train Loss: 0.028284370386004447 | Test Loss: 0.059444676452875135
[221/300]: Train Acc: 50.064 | Test Acc: 10.0 | Train Loss: 0.028284370386004447 | Test Loss: 0.059444676452875135
[222/300]: Train Acc: 50.064 | Test Acc: 10.0 | Train Loss: 0.028284370386004447 | Test Loss: 0.059444676452875135
[223/300]: Train Acc: 50.064 | Test Acc: 10.0 | Train Loss: 0.028284370386004447 | Test Loss: 0.059444676452875135
[224/300]: Train Acc: 50.064 | Test Acc: 10.0 | Train Loss: 0.028284370386004447 | Test Loss: 0.059444676452875135
[225/300]: Train Acc: 50.064 | Test Acc: 10.0 | Train Loss: 0.028284370386004447

[289/300]: Train Acc: 1.296 | Test Acc: 10.0 | Train Loss: 0.03685940633296966 | Test Loss: 0.03648617407083511
[290/300]: Train Acc: 1.296 | Test Acc: 10.0 | Train Loss: 0.03685940633296966 | Test Loss: 0.03648617407083511
[291/300]: Train Acc: 1.296 | Test Acc: 10.0 | Train Loss: 0.03685940633296966 | Test Loss: 0.03648617407083511
[292/300]: Train Acc: 1.296 | Test Acc: 10.0 | Train Loss: 0.03685940633296966 | Test Loss: 0.03648617407083511
[293/300]: Train Acc: 1.296 | Test Acc: 10.0 | Train Loss: 0.03685940633296966 | Test Loss: 0.03648617407083511
[294/300]: Train Acc: 1.296 | Test Acc: 10.0 | Train Loss: 0.03685940633296966 | Test Loss: 0.03648617407083511
[295/300]: Train Acc: 1.296 | Test Acc: 10.0 | Train Loss: 0.03685940633296966 | Test Loss: 0.03648617407083511
[296/300]: Train Acc: 1.296 | Test Acc: 10.0 | Train Loss: 0.03685940633296966 | Test Loss: 0.03648617407083511
[297/300]: Train Acc: 1.296 | Test Acc: 10.0 | Train Loss: 0.03685940633296966 | Test Loss: 0.0364861740

...

[277/300]: Train Acc: 1.296 | Test Acc: 10.0 | Train Loss: 0.03685940641403198 | Test Loss: 0.03652011765241623
[278/300]: Train Acc: 1.296 | Test Acc: 10.0 | Train Loss: 0.03685940641403198 | Test Loss: 0.03652011765241623
[279/300]: Train Acc: 1.296 | Test Acc: 10.0 | Train Loss: 0.03685940641403198 | Test Loss: 0.03652011765241623
[280/300]: Train Acc: 1.296 | Test Acc: 10.0 | Train Loss: 0.03685940641403198 | Test Loss: 0.03652011765241623
[281/300]: Train Acc: 1.296 | Test Acc: 10.0 | Train Loss: 0.03685940641403198 | Test Loss: 0.03652011765241623
[282/300]: Train Acc: 1.296 | Test Acc: 10.0 | Train Loss: 0.03685940641403198 | Test Loss: 0.03652011765241623
[283/300]: Train Acc: 1.296 | Test Acc: 10.0 | Train Loss: 0.03685940641403198 | Test Loss: 0.03652011765241623
[284/300]: Train Acc: 1.296 | Test Acc: 10.0 | Train Loss: 0.03685940641403198 | Test Loss: 0.03652011765241623
[285/300]: Train Acc: 1.296 | Test Acc: 10.0 | Train Loss: 0.03685940641403198 | Test Loss: 0.0365201176

[48/300]: Train Acc: 81.424 | Test Acc: 10.0 | Train Loss: 0.0261063116812706 | Test Loss: 0.1994288643836975
[49/300]: Train Acc: 81.424 | Test Acc: 10.0 | Train Loss: 0.026106312857866288 | Test Loss: 0.199428865814209
[50/300]: Train Acc: 81.424 | Test Acc: 10.0 | Train Loss: 0.02610631676197052 | Test Loss: 0.1994288556575775
[51/300]: Train Acc: 81.424 | Test Acc: 10.0 | Train Loss: 0.026106312314271927 | Test Loss: 0.1994288598060608
[52/300]: Train Acc: 81.424 | Test Acc: 10.0 | Train Loss: 0.02610631098985672 | Test Loss: 0.19942885723114015
[53/300]: Train Acc: 81.424 | Test Acc: 10.0 | Train Loss: 0.026106310625076293 | Test Loss: 0.1994288571357727
[54/300]: Train Acc: 81.424 | Test Acc: 10.0 | Train Loss: 0.02610631688952446 | Test Loss: 0.199428857088089
[55/300]: Train Acc: 81.424 | Test Acc: 10.0 | Train Loss: 0.0261063116812706 | Test Loss: 0.1994288643836975
[56/300]: Train Acc: 81.424 | Test Acc: 10.0 | Train Loss: 0.026106312857866288 | Test Loss: 0.199428865814209
[

[122/300]: Train Acc: 81.424 | Test Acc: 10.0 | Train Loss: 0.02610631098985672 | Test Loss: 0.19942885723114015
[123/300]: Train Acc: 81.424 | Test Acc: 10.0 | Train Loss: 0.026106310625076293 | Test Loss: 0.1994288571357727
[124/300]: Train Acc: 81.424 | Test Acc: 10.0 | Train Loss: 0.02610631688952446 | Test Loss: 0.199428857088089
[125/300]: Train Acc: 81.424 | Test Acc: 10.0 | Train Loss: 0.0261063116812706 | Test Loss: 0.1994288643836975
[126/300]: Train Acc: 81.424 | Test Acc: 10.0 | Train Loss: 0.026106312857866288 | Test Loss: 0.199428865814209
[127/300]: Train Acc: 81.424 | Test Acc: 10.0 | Train Loss: 0.02610631676197052 | Test Loss: 0.1994288556575775
[128/300]: Train Acc: 81.424 | Test Acc: 10.0 | Train Loss: 0.026106312314271927 | Test Loss: 0.1994288598060608
[129/300]: Train Acc: 81.424 | Test Acc: 10.0 | Train Loss: 0.02610631098985672 | Test Loss: 0.19942885723114015
[130/300]: Train Acc: 81.424 | Test Acc: 10.0 | Train Loss: 0.026106310625076293 | Test Loss: 0.199428

[195/300]: Train Acc: 50.064 | Test Acc: 10.0 | Train Loss: 0.0282843699580431 | Test Loss: 0.05972464237809181
[196/300]: Train Acc: 50.064 | Test Acc: 10.0 | Train Loss: 0.0282843699580431 | Test Loss: 0.05972464237809181
[197/300]: Train Acc: 50.064 | Test Acc: 10.0 | Train Loss: 0.0282843699580431 | Test Loss: 0.05972464237809181
[198/300]: Train Acc: 50.064 | Test Acc: 10.0 | Train Loss: 0.0282843699580431 | Test Loss: 0.05972464237809181
[199/300]: Train Acc: 50.064 | Test Acc: 10.0 | Train Loss: 0.0282843699580431 | Test Loss: 0.05972464237809181
[200/300]: Train Acc: 50.064 | Test Acc: 10.0 | Train Loss: 0.0282843699580431 | Test Loss: 0.05972464237809181
[201/300]: Train Acc: 50.064 | Test Acc: 10.0 | Train Loss: 0.0282843699580431 | Test Loss: 0.05972464237809181
[202/300]: Train Acc: 50.064 | Test Acc: 10.0 | Train Loss: 0.0282843699580431 | Test Loss: 0.05972464237809181
[203/300]: Train Acc: 50.064 | Test Acc: 10.0 | Train Loss: 0.0282843699580431 | Test Loss: 0.0597246423

[268/300]: Train Acc: 1.296 | Test Acc: 10.0 | Train Loss: 0.03685940800666809 | Test Loss: 0.03650301643610001
[269/300]: Train Acc: 1.296 | Test Acc: 10.0 | Train Loss: 0.03685940691947937 | Test Loss: 0.036503018927574155
[270/300]: Train Acc: 1.296 | Test Acc: 10.0 | Train Loss: 0.03685940675735474 | Test Loss: 0.03650301641225815
[271/300]: Train Acc: 1.296 | Test Acc: 10.0 | Train Loss: 0.036859406509399416 | Test Loss: 0.03650301605463028
[272/300]: Train Acc: 1.296 | Test Acc: 10.0 | Train Loss: 0.03685940639019013 | Test Loss: 0.03650301659107208
[273/300]: Train Acc: 1.296 | Test Acc: 10.0 | Train Loss: 0.036859406361579895 | Test Loss: 0.036503016233444215
[274/300]: Train Acc: 1.296 | Test Acc: 10.0 | Train Loss: 0.03685940639019013 | Test Loss: 0.03650301659107208
[275/300]: Train Acc: 1.296 | Test Acc: 10.0 | Train Loss: 0.03685940637588501 | Test Loss: 0.03650301659107208
[276/300]: Train Acc: 1.296 | Test Acc: 10.0 | Train Loss: 0.03685940634727478 | Test Loss: 0.036503

--- 

# BackUp

In [ ]:
resume = False
given_date = '20191113'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device == 'cuda':
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
### task: classification of the following classes
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

    
### hyperparameters
test_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch
num_epochs = 300 # number of iterations the model gets trained
learning_rate = 0.1 # factor for weight updates
learning_rate_switches = { # learning rate is reset after specific epochs
    '150': 0.01,
    '250': 0.001
}
momentum = 0.9
weight_decay = 5e-4

start = timeit.default_timer()

rows = []
for seed in SEEDS:
    for shuffle_setting in [{'train': False, 'test': False}]:#, {'train': True, 'test': True}]:
        print("\n==> Beginning training with seed {} and shuffle setting {}".format(seed, shuffle_setting))
        print("-" * 30)
        torch.manual_seed(seed)
        if device == 'cuda':
            torch.cuda.manual_seed_all(seed)
             
        ### Model
        print('==> Building model..')
        net = ResNet18()
        net = net.to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=momentum, weight_decay=weight_decay)
        
        ### load the data
        # if needed, specify batch sizes and shuffle settings
        print('==> Loading data..')
        dataloader = DataLoader()
        dataloader.download_cifar()
        dataloader.prepare_cifar('homogeneous', random_state=seed, batch_size=64)

        if resume:
            print('==> Resuming from checkpoint..')
            assert os.path.isdir('serialized'), 'Error: no serialized directory found!'
            ckpt = torch.load('./serialized/{}/ckpt_{}.pth'.format(given_date, seed))
            test_acc, start_epoch, net = net.load(ckpt)

        for epoch in range(start_epoch, start_epoch+num_epochs):
            if str(epoch+1) in learning_rate_switches.keys():
                print('==> Resetting learning rate')
                for param_group in optimizer.param_groups:
                    param_group['lr'] = learning_rate_switches[str(epoch+1)]
                    
            train_acc, train_loss = train(epoch, optimizer, criterion, dataloader)
            test_acc, test_loss = test(epoch, test_acc, seed, dataloader)
            print("[{}/{}]: Train Acc: {} | Test Acc: {} | Train Loss: {} | Test Loss: {}"\
                  .format(epoch+1, num_epochs, train_acc, test_acc, train_loss, test_loss))
            row = {
                'epoch': epoch + 1,
                'seed': seed,
                'train': True,
                'shuffle': True if shuffle_setting['train'] == True else False,
                'accuracy': train_acc,
                'loss': train_loss
            }
            rows.append(row)
            row = {
                'epoch': epoch + 1,
                'seed': seed,
                'train': False,
                'shuffle': True if shuffle_setting['train'] == True else False,
                'accuracy': test_acc,
                'loss': test_loss
            }
            rows.append(row)
            
stop = timeit.default_timer()
logging_df = pd.DataFrame(rows, columns=['epoch', 'seed', 'train', 'shuffle', 'accuracy', 'loss'])   
training_logs_dir = 'training_logs'
logging_df.to_csv('{}.txt'.format(os.path.join(training_logs_dir, today)), sep='\t', index=False)
